In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tqdm import tqdm

import numpy as np
import pandas as pd

import re

import matplotlib.pyplot as plt

import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
# from torch.nn.functional import relu, sigmoid
from torch import optim

import torch
from torch.utils.data import Dataset

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/captions.txt')

In [ ]:
df.head()

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...


In [ ]:

def clean_caption(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = " ".join([w for w in text.split() if len(w) > 1])
    return text

df['caption'] = df['caption'].apply(clean_caption)


In [ ]:
captions = df["caption"].tolist()
max_length = max(len(caption.split()) for caption in captions)
max_length

32

In [ ]:
# captions

In [ ]:
print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  Tesla T4
Using device: cuda


In [ ]:
features1 = pd.read_csv('/content/drive/MyDrive/final1_feature.csv')

In [ ]:
features1.head(5)

In [ ]:
features = pd.read_csv('/content/drive/MyDrive/features.csv')

In [ ]:
features.head(5)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,1000268201_693b08cb0e.jpg,0.000530,0.020955,0.000000,0.000478,0.102640,2.816984,0.000314,0.000894,0.500193,...,5.755997,0.000000,11.968189,0.855762,2.582031,0.000000,2.545345,3.693776,3.893597,5.397244
1,1001773457_577c3a7d70.jpg,0.000896,0.004986,0.001943,0.005952,0.012798,0.276479,0.001835,0.012086,0.115777,...,0.703914,5.111288,5.492939,10.403438,0.000000,0.000000,1.604755,3.473159,0.753371,13.305740
2,1002674143_1b742ab4b8.jpg,0.000139,0.010884,0.000000,0.001002,0.310223,0.572058,0.001189,0.001960,0.230935,...,2.796671,0.000000,2.526081,9.665098,3.163004,0.000000,4.403409,0.000000,2.451269,15.814799
3,1003163366_44323f5815.jpg,0.000156,0.017315,0.001471,0.002552,0.038932,1.582965,0.001269,0.003844,0.227926,...,1.709077,2.349216,6.488191,12.943340,0.000000,0.000000,3.434384,2.654669,10.393051,2.786068
4,1007129816_e794419615.jpg,0.000144,0.001897,0.008756,0.018821,0.000000,0.000000,0.002928,0.002050,0.459453,...,2.115044,4.440852,8.961600,5.716276,2.507264,2.447746,0.538589,0.000000,7.990150,2.681692


In [ ]:
result_dict = features.to_dict(orient='index')
output_dict = {}
for key, value in result_dict.items():
    image_name = value['Unnamed: 0']
    features = np.array([value[str(i)] for i in range(len(value) - 1)])
    output_dict[image_name] = features.reshape(1, -1)
# print(output_dict)

In [ ]:
sample_feat = next(iter(output_dict.values()))  # (1, F)
IMG_FEAT_DIM = sample_feat.shape[-1]
print("Image‑feature dim =", IMG_FEAT_DIM)

Image‑feature dim = 1024


In [ ]:
from torch.utils.data import Dataset
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
import numpy as np

class BERTCaptionDataset(Dataset):
    def __init__(self, df, image_features, tokenizer, max_length=30):
        self.df = df.reset_index(drop=True)
        self.image_features = image_features
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_id = row["image"]
        caption = row["caption"]

        # ویژگی‌های تصویر
        img_feat = self.image_features[image_id]
        if isinstance(img_feat, np.ndarray) and len(img_feat.shape) == 2 and img_feat.shape[0] == 1:
            img_feat = img_feat.squeeze(0)
        img_feat = torch.tensor(img_feat, dtype=torch.float32)

        # توکنایز کپشن
        tokens = self.tokenizer.encode(caption, add_special_tokens=False)
        input_tokens = tokens[:-1]
        target_tokens = tokens[1:]

        input_tokens_padded = pad_sequences([input_tokens], maxlen=self.max_length, padding='post')[0]
        target_tokens_padded = pad_sequences([target_tokens], maxlen=self.max_length, padding='post')[0]

        input_tensor = torch.tensor(input_tokens_padded, dtype=torch.long)
        target_tensor = torch.tensor(target_tokens_padded, dtype=torch.long)

        return (img_feat, input_tensor), target_tensor


In [ ]:
# class BERTCaptionDataset(Dataset):
#     def __init__(self, image_features, captions, tokenizer):
#         self.image_features = image_features
#         self.captions = captions  # list of (image_id, caption)
#         self.tokenizer = tokenizer

#     def __len__(self):
#         return len(self.captions)

#     def __getitem__(self, idx):
#         image_id, caption = self.captions[idx]
#         img_feat = torch.tensor(self.image_features[image_id], dtype=torch.float).squeeze(0)

#         encoded = self.tokenizer(caption, padding='max_length', truncation=True, max_length=30, return_tensors="pt")
#         input_ids = encoded["input_ids"].squeeze(0)
#         attention_mask = encoded["attention_mask"].squeeze(0)

#         return (img_feat, attention_mask), input_ids


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
image_ids = list(output_dict.keys())
# captions_with_ids = list(zip(image_ids, captions))
dataset = BERTCaptionDataset(df,output_dict, tokenizer)


In [ ]:
from torch.utils.data import DataLoader

# train_dataset = BERTCaptionDataset(df, output_dict, tokenizer)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)




In [ ]:
for (X1, X2), y in train_loader:
    print("🔹 Image feature batch shape:", X1.shape)  # [batch_size, feature_dim]
    print("🔹 Attention mask batch shape:", X2.shape) # [batch_size, seq_len]
    print("🔹 Input ids batch shape:", y.shape)       # [batch_size, seq_len]
    break

🔹 Image feature batch shape: torch.Size([8, 1024])
🔹 Attention mask batch shape: torch.Size([8, 30])
🔹 Input ids batch shape: torch.Size([8, 30])


In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

# بارگذاری مدل از پیش آموزش‌دیده BERT
bert = BertModel.from_pretrained("bert-base-uncased")
bert_emb = bert.embeddings.word_embeddings

class BERTUniLSTMResidual(nn.Module):
    def __init__(self, img_feat_dim, embed_dim, hidden_dim,
                 vocab_size, max_len, dropout_rate=0.5):
        super().__init__()

        self.img_fc = nn.Linear(img_feat_dim, embed_dim)
        self.embedding = bert_emb
        self.embedding.weight.requires_grad_(False)

        self.lstm = nn.LSTM(
            input_size=embed_dim * 2,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=False
        )

        self.res_proj = nn.Linear(embed_dim * 2, hidden_dim, bias=False)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, img_feat, input_ids):
        img_emb = self.img_fc(img_feat)
        txt_emb = self.embedding(input_ids)
        seq_len = input_ids.size(1)

        img_emb_expanded = img_emb.unsqueeze(1).expand(-1, seq_len, -1)
        combined = torch.cat([txt_emb, img_emb_expanded], dim=-1)

        lstm_out, _ = self.lstm(combined)
        out = lstm_out + self.res_proj(combined)  # residual connection
        out = self.dropout(out)
        return self.fc(out)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
vocab_size = tokenizer.vocab_size
model = BERTUniLSTMResidual(
    img_feat_dim=IMG_FEAT_DIM,
    embed_dim=bert_emb.embedding_dim,
    hidden_dim=512,
    vocab_size=tokenizer.vocab_size,
    max_len=max_length
)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# optimizer = optim.Adam(model.parameters(), lr=1e-4)
# criterion = nn.CrossEntropyLoss()

BERTUniLSTMResidual(
  (img_fc): Linear(in_features=1024, out_features=768, bias=True)
  (embedding): Embedding(30522, 768, padding_idx=0)
  (lstm): LSTM(1536, 512, batch_first=True)
  (res_proj): Linear(in_features=1536, out_features=512, bias=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=30522, bias=True)
)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-2)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                       patience=2, factor=0.3)
criterion  = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

num_epochs = 10
clip_value = 5.0

for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0

    for (img_feat, input_ids), target in tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}"):
        img_feat, input_ids, target = img_feat.to(device), input_ids.to(device), target.to(device)

        optimizer.zero_grad()
        logits = model(img_feat, input_ids)               # (B, L, vocab)
        logits = logits.view(-1, logits.size(-1))         # (B*L, vocab)
        target = target.view(-1)                          # (B*L,)

        loss = criterion(logits, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch}: loss = {avg_loss:.4f}")
    scheduler.step(avg_loss)


Epoch 1/10: 100%|██████████| 5057/5057 [02:00<00:00, 41.92it/s]


Epoch 1: loss = 5.4091


Epoch 2/10: 100%|██████████| 5057/5057 [01:56<00:00, 43.39it/s]


Epoch 2: loss = 4.5518


Epoch 3/10: 100%|██████████| 5057/5057 [01:56<00:00, 43.54it/s]


...

Epoch 8: loss = 3.7514


Epoch 9/10: 100%|██████████| 5057/5057 [01:55<00:00, 43.60it/s]


Epoch 9: loss = 3.7060


Epoch 10/10: 100%|██████████| 5057/5057 [01:56<00:00, 43.38it/s]

Epoch 10: loss = 3.6615


In [ ]:
import torch
import torch.nn.functional as F

def generate_caption(
    model,
    tokenizer,
    image_feature,          # numpy array یا torch.Tensor  با ابعاد (1, F) یا (F,)
    device="cpu",
    max_length=30,
    top_k=1                 # =1 ⟺ greedy؛ عدد بزرگ‌تر ⇒ sampling
):
    """
    خروجی: رشتهٔ کپشن (بدون [CLS]/[SEP])
    """
    model.eval()

    # 1) آماده‌سازی فیچر تصویر → Tensor (1, F)
    if isinstance(image_feature, torch.Tensor):
        img = image_feature.squeeze()
    else:                              # numpy یا لیست
        img = torch.tensor(image_feature, dtype=torch.float32).squeeze()
    if img.dim() == 1:
        img = img.unsqueeze(0)         # (1, F)
    img = img.to(device)

    # 2) دنبالهٔ اولیه با [CLS]
    seq_ids = [tokenizer.cls_token_id or 101]

    with torch.no_grad():
        for _ in range(max_length):
            seq_tensor = torch.tensor([seq_ids],
                                      dtype=torch.long,
                                      device=device)          # (1, L)

            # 3) پیش‌بینی توکن بعدی
            logits = model(img, seq_tensor)                   # (1, L, vocab)
            next_logits = logits[:, -1, :]                    # (1, vocab)

            if top_k == 1:                                    # Greedy
                next_id = torch.argmax(next_logits, dim=-1).item()
            else:                                             # Top‑k sampling
                vals, idxs = torch.topk(next_logits, k=top_k, dim=-1)
                probs = F.softmax(vals, dim=-1)
                next_id = idxs[0, torch.multinomial(probs, 1)].item()

            # پایان
            if next_id == (tokenizer.sep_token_id or 102):
                break

            seq_ids.append(next_id)

    # 4) تبدیل آیدی‌ها به جمله (بدون CLS/SEP)
    caption = tokenizer.decode(seq_ids[1:], skip_special_tokens=True)
    return caption


In [ ]:
test_feat = output_dict["23445819_3a458716c1.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


puppies in its mouth wide open in the grass in the grass with his mouth open in its mouth open in its mouth open field in the grass and


In [ ]:
test_feat = output_dict["23445819_3a458716c1.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


collie catches tennis ball in its mouth in the grass with his puppies its mouth wide open field in its mouth open to catch red ball in the


In [ ]:
test_feat = output_dict["23445819_3a458716c1.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


puppies in the grass with his puppies in the grass with his mouth wide open in the grass with white dog in the grass with his mouth open


In [ ]:
test_feat = output_dict["23445819_3a458716c1.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


puppies and white dog in the grass and white dog in the grass in field of the grass and puppies in the grass and white dog catching the


In [ ]:
test_feat = output_dict["103106960_e8a41d64f8.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


goalie in the air on the ice skating on the goalie in the goal goal goal in the background of goalie in the background watches from the


In [ ]:
test_feat = output_dict["131632409_4de0d4e710.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


floral pauses pauses stuffed animalside stuffed animal earring towel towel towelcanck knife towelcan towelesbee thrown towel towel toweles stuffed animal towelcan


In [ ]:
test_feat = output_dict["12830823_87d2654e31.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


##s and child in blue shorts and white shirt and backpacks are playing in the street with the water and watch from the other people watch them in the


In [ ]:
test_feat = output_dict["10815824_2997e03d76.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


##r is riding horses on the ground with his mouth open in the background in field of the background of them fire on the ground with the other people in


In [ ]:
test_feat = output_dict["49553964_cee950f3ba.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


waterskier is pulled up in the water with stick in the water and the ocean on the water while the ocean in the water and red paddles


In [ ]:
test_feat = output_dict["113678030_87a6a6e42e.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)

skier in white jacket is jumping on the snow covered mountain in front of snowboard on the snow covered mountain range in the background and white skis down


In [ ]:
test_feat = output_dict["166654939_80ea4ddbcc.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


cars racing down dirt road in front of the street with cars in the background with cars and cars in front of the horizon with the background is in the


In [ ]:
test_feat = output_dict["221973402_ecb1cd51f1.jpg"]   # ← یک آرایهٔ (1, F)
caption = generate_caption(model, tokenizer, test_feat,
                           device=device, max_length=30, top_k=3)
print(caption)


girl with red shirt and red shirt is walking down the grass with his dog in red shirt is jumping on the water with red frisbee on his
